<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/Dataframe_1_(Practical1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# load the libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [7]:
# create spark session
sc : SparkContext = SparkSession\
.builder\
.appName("IMDB Analysis")\
.master('local')\
.getOrCreate()\
.sparkContext

In [12]:
# Import datea
#movies=sc.textFile("movies.dat")
from google.colab import files
movies=files.upload()


Saving movies.dat to movies.dat


dict

In [16]:
# Create Rdd
movies=sc.textFile("movies.dat");
movies.take(3)

["1::Toy Story (1995)::Animation|Children's|Comedy",
 "2::Jumanji (1995)::Adventure|Children's|Fantasy",
 '3::Grumpier Old Men (1995)::Comedy|Romance']

In [17]:
# split the string with double ::
def splitString(myfile):
  return myfile.split("::",4)

In [19]:
ratingFile=files.upload()

Saving ratings.dat to ratings.dat


In [23]:
movies=sc.textFile("movies.dat").map(splitString);
rating=sc.textFile("ratings.dat").map(splitString)

Display text from datea sets

In [27]:
movies.take(3)


[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"],
 ['2', 'Jumanji (1995)', "Adventure|Children's|Fantasy"],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance']]

In [25]:
rating.take(3)

[['1', '1193', '5', '978300760'],
 ['1', '661', '3', '978302109'],
 ['1', '914', '3', '978301968']]